In [6]:
import cv2
import numpy as np
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import time
import os
import random
from scipy import stats

SEG_MODEL_PATH = r"..\final\segmentation_model.pt"
CLS_MODEL_PATH = r"..\final\classification_model.pt"

seg_model = YOLO(SEG_MODEL_PATH)
cls_model = YOLO(CLS_MODEL_PATH)


Ultralytics YOLOv8.2.103  Python-3.10.0 torch-2.5.1+cpu CPU (Intel Core(TM) i5-8250U 1.60GHz)
Setup complete  (8 CPUs, 7.9 GB RAM, 141.9/145.9 GB disk)


In [7]:
def SR_remove(image):
    start = time.time()
    orig = image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    _, image = cv2.threshold(image, 235, 255, cv2.THRESH_BINARY)
    mask = image
    srs = cv2.bitwise_and(orig, orig, mask=mask)
    sub = orig - srs
    lower_val = np.array([0,0,0])
    upper_val = np.array([50,50,50])
    hsv = cv2.cvtColor(sub, cv2.COLOR_BGR2HSV)
    black_mask = cv2.inRange(hsv, lower_val, upper_val)
    kernel = np.ones((9,9), np.uint8)
    dilated = cv2.dilate(black_mask, kernel, iterations = 1)
    final = cv2.inpaint(orig, dilated, inpaintRadius=30, flags=cv2.INPAINT_TELEA)
    return final, time.time() - start

In [8]:
def apply_segment(image):
    start = time.time()
    results = seg_model.predict(image, verbose = False)
    for result in results:
            height, width = result.orig_img.shape[:2]
            background = np.ones((height, width, 3), dtype=np.uint8) * 255
            masks = result.masks.xy
            orig_img = result.orig_img
            for mask in masks:
                    mask = mask.astype(int)
                    mask_img = np.zeros_like(orig_img)
                    cv2.fillPoly(mask_img, [mask], (255, 255, 255))
                    masked_object = cv2.bitwise_and(orig_img, mask_img)
                    background[mask_img == 255] = masked_object[mask_img == 255]
    return background, time.time() - start

In [9]:
def get_classification(img):
    start = time.time()
    results = cls_model.predict(img, verbose = False)
    for result in results:
          id = result.probs.top1
          name = result.names[id]
    return name, time.time() - start

In [10]:
def predict(img):
     sr, sr_time = SR_remove(img)
     segmented, seg_time = apply_segment(sr)
     pred, cls_time = get_classification(segmented)
     print(sr_time, seg_time, cls_time, pred, sr_time+seg_time+cls_time,"seconds")

In [11]:
path = r"path_to_data"
image_paths = list(os.listdir(path))
sr_times, seg_times, cls_times = [], [], []
total_times = []
for i in range(100):
    image = random.choice(image_paths)
    image_paths.remove(image)
    img_path = os.path.join(path, image)
    print(image)
    start = time.time()
    sr, sr_time = SR_remove(cv2.imread(img_path))
    segmented, seg_time = apply_segment(sr)
    pred, cls_time = get_classification(segmented)
    total = time.time() - start
    total_times.append(total)
    sr_times.append(sr_time)
    seg_times.append(seg_time)
    cls_times.append(cls_time)
    print(sr_time, seg_time, cls_time, pred, sr_time+seg_time+cls_time,"seconds", total)

AABV1.jpg
4.110381603240967 0.4325118064880371 0.39897656440734863 CIN2+ 4.9418699741363525 seconds 4.975194215774536
AAEW2.jpg
7.3148651123046875 0.05759382247924805 0.06161689758300781 CIN2+ 7.434075832366943 seconds 7.455425024032593
AAFQ3.jpg
2.7562456130981445 0.0667412281036377 0.05812478065490723 CIN2+ 2.8811116218566895 seconds 2.9009172916412354
AACD6.jpg
7.5860915184021 0.06569170951843262 0.05845165252685547 CIN2+ 7.710234880447388 seconds 7.73112154006958
AABG1.jpg
6.648000717163086 0.062184810638427734 0.05682849884033203 CIN2+ 6.767014026641846 seconds 6.792881488800049
AACT00.jpg
2.8539788722991943 0.06065773963928223 0.05822587013244629 CIN2+ 2.972862482070923 seconds 2.9826276302337646
AABW1.jpg
3.3559086322784424 0.06209421157836914 0.0626211166381836 CIN2+ 3.480623960494995 seconds 3.5050034523010254
AEH0.jpg
8.910057306289673 0.06586766242980957 0.06171989440917969 CIN1- 9.037644863128662 seconds 9.050743579864502
ABK0.jpg
3.691584587097168 0.06160545349121094 0.063

In [12]:
print("SR removal:",stats.describe(sr_times))
print(stats.describe(sr_time).mean)
print("Segmentation:",stats.describe(seg_times))
print("Prediction:",stats.describe(cls_times))
print("Total time:",stats.describe(total_times))

SR removal: DescribeResult(nobs=100, minmax=(1.039588451385498, 27.398273706436157), mean=6.50673330783844, variance=20.13396556182636, skewness=2.1059058291595063, kurtosis=5.734133762368648)
4.588156700134277
Segmentation: DescribeResult(nobs=100, minmax=(0.04789996147155762, 0.4325118064880371), mean=0.06826021432876588, variance=0.0014164705607179128, skewness=9.19753106354792, kurtosis=86.53193788570448)
Prediction: DescribeResult(nobs=100, minmax=(0.05010533332824707, 0.39897656440734863), mean=0.06556020498275757, variance=0.0013143225659950238, skewness=8.249642457709605, kurtosis=71.16925661114345)
Total time: DescribeResult(nobs=100, minmax=(1.2070763111114502, 27.543017625808716), mean=6.663960018157959, variance=20.15566384767599, skewness=2.1053996680414357, kurtosis=5.723965002262524)


c:\Users\taral\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:1087: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
c:\Users\taral\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:1087: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
